# Affordable rental units
#### Author: Francesco Ciraolo

## Dataset origin
https://www.bostonplans.org/housing/finding-housing/property-listings

## Goals
 - Extracting the rental units table, explode by layouts and associate the positions

### Transversal Goals
#### For deliverable 1
 - Match with most common violations


In [1]:
import os
import pandas as pd
import re

In [10]:
from geopy import Nominatim
class AddressToCoordinates:

    def __init__(self, user_agent='undefined'):
        self.user_agent = user_agent
        self.geolocator = Nominatim(user_agent=user_agent)

    def get_coordinates(self, address):
        location = self.geolocator.geocode(address)
        return (location.latitude, location.longitude) if location else (None, None)

atc = AddressToCoordinates()

In [39]:

source_path = '../data/affordable_rental_units.csv'

if not os.path.exists(source_path):
    from bs4 import BeautifulSoup
    import requests

    source_url = 'https://www.bostonplans.org/housing/finding-housing/property-listings'
    page_text = requests.get(source_url).text
    del source_url

    soup = BeautifulSoup(page_text, 'html.parser')
    del page_text


    spreadsheet_url = soup.find_all('iframe')[-1].get('src')
    spreadsheet_url = '/'.join(str(spreadsheet_url).split('/')[:-1] + ['pub?output=csv'])

    spreadsheet_content = requests.get(spreadsheet_url, allow_redirects=True).text
    spreadsheet_content = re.sub('â\x80\x93', '-', spreadsheet_content)

    with open(source_path, 'w') as file:
        file.write(spreadsheet_content)

    del spreadsheet_url
    del file
    del spreadsheet_content

source_pd = pd.read_csv(source_path, skiprows=1)
# Rename columns
source_pd.columns = ['Neighborhood',
                     'Address',
                     'Info',
                     'Units',
                     'Sizes',
                     'AMI']
#Drop info
source_pd = source_pd[['Neighborhood',
                     'Address',
                     'Units',
                     'Sizes',
                     'AMI']]

# Weak assumption
source_pd['Units'] = source_pd['Units'].fillna(1)

source_pd = source_pd.dropna()

def explode_address(address):
    _old = address
    if '\n' in address:
        _complex, address = address.split('\n')
    else:
        m = re.search(r'^(?:(.*?)??[ ,]*\s)?(\d+(?:\s*[-&]\s*\d+)?\s*.+)', address)
        if m:
            _complex = m.groups()[0] if len(m.groups()) > 1 else None
            address = m.groups()[1]
      #      print(f'M Complex: {str(_complex):30} Address: {str(address):30} Old: {_old}\n{m.groups()}\n')
        else:
            _complex = address
            address = None
     #       print(f'N Complex: {_complex:30} Address: {str(address):30} Old: {_old}\n')


    #return _complex, address, None, None
    if _complex:
        lat, lon = atc.get_coordinates(_complex + ' Boston')
    else:
        lat = None

    if not lat:
        if address:
            m = re.search('(\d+)\s*[-&]\s*\d+\s*(.*)', address)
            if m:
                addr = f'{m.group(1)} {m.group(2)}'
            else:
                addr = address

        if address:
            lat, lon = atc.get_coordinates(addr + ' Boston')

    if not lat:
        lat = lon = None

    return _complex, address, lat, lon

#source_pd[['Complex', 'Address']] = source_pd.apply(lambda r: (None, r['Address']) if not '\n' in r['Address'] else r['Address'].split('\n'), axis='columns', result_type='expand')
source_pd[['Complex', 'Address', 'Latitude', 'Longitude']] = source_pd.apply(lambda r: explode_address(r['Address']), axis='columns', result_type='expand')


source_pd

,Neighborhood,Address,Units,Sizes,AMI,Complex,Latitude,Longitude
0,Allston,8 Griggs Street,2.0,1BR,80-100%; 100%,None,42.348330,-71.133503
1,Allston,31 North Beacon Street,3.0,"Studio, 1BR, 2BR",70%,None,42.353909,-71.138789
2,Allston,40 Malvern Street,6.0,"1BR, 2BR",70%,None,42.354018,-71.125015
3,Allston,450 Cambridge Street,5.0,"1BR, 2BR",70%,None,42.353926,-71.135720
4,Allston,1505 Commonwealth Avenue,10.0,"Studio, 1BR, 2BR",70%,None,42.347084,-71.141588
...,...,...,...,...,...,...,...,...
112,West End,None,30.0,"Studio, 1BR, 2BR, 3BR","80%, 80-120%",Avalon North Station,42.365964,-71.063249
113,West End,1 & 10 Emerson Place,19.0,"Studio, 1BR, 2BR",90%,Emerson Place,42.364165,-71.068558
114,West Roxbury,446 Belgrade Avenue,5.0,"Studio, 1BR, 2BR, 3BR",70%,Belgrade Place,42.286486,-71.150155
115,West Roxbury,1215 Centre Street,12.0,1BR,80%,Residences at Snow Place,42.298188,-71.131232


In [41]:
atc.get_coordinates('Alyx Boston')

(None, None)

In [123]:
from address_to_coordinates import AddressToCoordinates as BaseATC

class AddressToCoordinates(BaseATC):

    def __init__(self):
        super().__init__(user_agent="SparkBU")

    def get_coordinates(self, address):
        chunks = address.split('\n')
        chunks = []


atc = BaseATC()
#atc.get_coordinates(source_pd.loc[47]['Address'].split('\n')[-1].split('@')[-1] + ',')
atc.get_coordinates('Avalon @ Prudential, Boston, MA')

AttributeError: 'NoneType' object has no attribute 'latitude'